In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(atom="O 0 0 0;H 0.758602 0 0.504284;H 0.758602 0 -0.504284",basis="sto-3g")
problem = driver.run()

In [3]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
converter = QubitConverter(JordanWignerMapper())
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
ansatz = UCCSD()
ansatz.num_particles = problem.num_particles
ansatz.num_spatial_orbitals = problem.num_spatial_orbitals
ansatz.qubit_converter = converter
initial_state = HartreeFock()
initial_state.num_particles = problem.num_particles
initial_state.num_spatial_orbitals = problem.num_spatial_orbitals
initial_state.qubit_converter = converter
ansatz.initial_state = initial_state

In [4]:
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

In [7]:
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
adapt_vqe = AdaptVQE(vqe,max_iterations=100,threshold=1e-3)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

In [8]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(converter, adapt_vqe)
result = solver.solve(problem)

KeyboardInterrupt: 